In [ ]:
!pip install "git+https://github.com/nixtla/neuralforecast.git@main"
!pip install darts

In [ ]:
# import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss

from datetime import date
from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.utils.statistics import check_seasonality, extract_trend_and_seasonality
from darts.metrics import mape, rmse, mae, smape
from darts.utils.timeseries_generation import datetime_attribute_timeseries, holidays_timeseries
from darts.utils.likelihood_models import QuantileRegression

In [ ]:
Y_df = pd.read_csv('Y_df.csv')

Y_df['datetime'] = pd.to_datetime(Y_df['ds'])
Y_df.drop('ds', axis=1, inplace=True)
Y_df.set_index('datetime', inplace=True)
series = TimeSeries.from_series(Y_df)

series = series.add_datetime_attribute('hour')
series = series.add_datetime_attribute('dayofweek')
series = series.add_datetime_attribute('month')
series = series.add_datetime_attribute('quarter')
series = series.add_datetime_attribute('day')
series = series.add_datetime_attribute('year')
series = series.add_holidays(country_code='ITA')

Y_df = TimeSeries.pd_dataframe(series).reset_index()
Y_df.rename({'datetime': 'ds'}, axis=1, inplace=True)
Y_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
Y_df.reset_index(inplace=True)
def is_bridge_day(day, holiday, dayofweek):
    if holiday == 1:
        return 7
    elif dayofweek in [5,6] and holiday == 0:
        return dayofweek
    elif dayofweek == 0 and holiday == 0 and Y_df.iloc[day+24]['holidays'] == 1:
        return 8
    elif dayofweek == 4 and holiday == 0 and Y_df.iloc[day-24]['holidays'] == 1:
        return 8
    else:
        return dayofweek

Y_df['day_of_week'] = np.vectorize(is_bridge_day)(Y_df.index, Y_df['holidays'], Y_df['dayofweek'])

encoder = LabelEncoder()

calendar = ['hour', 'day', 'day_of_week', 'month', 'year']
for cal in calendar:
  Y_df[cal] = encoder.fit_transform(Y_df[cal]).astype(np.int32)
Y_df

In [ ]:
val_size  = 190*24
test_size = 190*24
Y_train_df = Y_df.iloc[:-test_size, :]
Y_test_df = Y_df.iloc[-test_size:, :]
Y_val_df = Y_train_df.iloc[-val_size:, :]

In [ ]:
# Create the plot
import matplotlib
matplotlib.rc_file_defaults()

# Create subplots with two rows
fig, axs = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Plot the load data in the first subplot
axs[0].plot(Y_train_df['ds'], Y_train_df['y'], color='steelblue', label='Electricity Price')
axs[0].plot(Y_val_df['ds'], Y_val_df['y'], color='steelblue')
axs[0].plot(Y_test_df['ds'], Y_test_df['y'], color='steelblue')
axs[0].set_ylabel('Spot Price (€/MWh)', fontsize=12)
axs[0].legend(loc = "upper left")
legend = axs[0].get_legend()

# Set the font size of the legend
legend.get_frame().set_facecolor('white')  # Optional: Set legend background color
legend.get_frame().set_linewidth(0.5)  # Optional: Set legend frame linewidth
for text in legend.get_texts():
    text.set_fontsize(12)  # Set the font size


# Plot the temperature data in the second subplot
axs[1].plot(Y_train_df['ds'], Y_train_df['psvda'], color='seagreen', label='PSVDA')
axs[1].plot(Y_val_df['ds'], Y_val_df['psvda'], color='seagreen')
axs[1].plot(Y_test_df['ds'], Y_test_df['psvda'], color='seagreen')
axs[1].set_ylabel('PSV day-ahead (€/MWh)', fontsize=12)
axs[1].legend(loc="upper left")
# Get the current legend
legend_1 = axs[1].get_legend()

# Set the font size of the legend
legend_1.get_frame().set_facecolor('white')  # Optional: Set legend background color
legend_1.get_frame().set_linewidth(0.5)  # Optional: Set legend frame linewidth
for text in legend_1.get_texts():
    text.set_fontsize(12)  # Set the font size

# Plot the temperature data in the second subplot
axs[2].plot(Y_train_df['ds'], Y_train_df['load forecast'], color='lightcoral', label='Load Forecast')
axs[2].plot(Y_val_df['ds'], Y_val_df['load forecast'], color='lightcoral')
axs[2].plot(Y_test_df['ds'], Y_test_df['load forecast'], color='lightcoral')
axs[2].set_ylabel('Load (MW)', fontsize=12)
axs[2].legend(loc="upper left")
# Get the current legend
legend_2 = axs[2].get_legend()

# Set the font size of the legend
legend_2.get_frame().set_facecolor('white')  # Optional: Set legend background color
legend_2.get_frame().set_linewidth(0.5)  # Optional: Set legend frame linewidth
for text in legend_2.get_texts():
    text.set_fontsize(12)  # Set the font size


# Add annotations for the splits
axs[0].annotate('Training', xy=(Y_train_df['ds'].mean(), Y_train_df['y'].max()), xytext=(0, 20),
             xycoords='data', textcoords='offset points', fontsize=15, ha='center')
axs[0].annotate('Validation', xy=(Y_val_df['ds'].mean(), Y_train_df['y'].max()), xytext=(0, 20),
             xycoords='data', textcoords='offset points', fontsize=15, ha='center')
axs[0].annotate('Test', xy=(Y_test_df['ds'].mean(), Y_train_df['y'].max()), xytext=(0, 20),
             xycoords='data', textcoords='offset points', fontsize=15, ha='center')

# Add dashed lines for the splits
axs[0].axvline(Y_val_df['ds'].iloc[0], color='k', linestyle='--')
axs[0].axvline(Y_val_df['ds'].iloc[-1], color='k', linestyle='--')
axs[1].axvline(Y_val_df['ds'].iloc[0], color='k', linestyle='--')  # Shared vertical line
axs[1].axvline(Y_val_df['ds'].iloc[-1], color='k', linestyle='--')
axs[2].axvline(Y_val_df['ds'].iloc[0], color='k', linestyle='--')  # Shared vertical line
axs[2].axvline(Y_val_df['ds'].iloc[-1], color='k', linestyle='--')

plt.xlabel('Datetime', fontsize=12)
#plt.suptitle('Electricity Demand', fontsize=15)
plt.subplots_adjust(hspace=0)
plt.show()

In [ ]:

# Create a copy of Y_train_df
df_train = Y_train_df.copy()

# Map quarter labels to the "Quarter" column
quarter_labels = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}
df_train['Quarter'] = df_train['quarter'].map(quarter_labels)

# Create the plot
plt.figure(figsize=(10, 7))
sns.lineplot(data=df_train, x='hour', y='y', hue='Quarter', palette=palette, linewidth=3)

# Remove the spines
sns.despine()

# Set the title
plt.ylabel('Price (€/MWh)')

# Change the legend labels
legend = plt.legend(fontsize='large')
for line, label in zip(legend.get_lines(), quarter_labels.values()):
    line.set_linewidth(3.0)
    line.set_label(label)
plt.grid(False)
# Show the plot
plt.show()


In [ ]:
Y_df.set_index('ds', inplace=True)

In [ ]:
df_4 = Y_df[['y', 'hour', 'dayofweek', 'month', 'holidays', 'day', 'year', 'psvda', 'load forecast']]

# Suppose you have a DataFrame called 'df' containing the relevant data including 'hour_profile' column

# Create past regressor columns
num_past_periods = 168  # Number of past observed periods

# past gas
df_4[f'psvda_t-{24}'] = Y_df['psvda'].shift(24)

# target
for i in range(1, 24):
    df_4[f'y_t+{i}'] = Y_df['y'].shift(-i)

# future load
for i in range(0, 24):
    df_4[f'load_forecast_t+{i}'] = Y_df['load forecast'].shift(-i)

for i in range(1, 169):
    df_4[f'y_t-{i}'] = Y_df['y'].shift(i)

df_4.rename({'y': 'y_t+0'}, axis=1, inplace=True)
# Remove rows with NaN values due to shifting
df_4 = df_4.dropna()
df_4['weekday-hour'] = df_4['dayofweek']*df_4['hour']
df_4['month-hour'] = df_4['month']*df_4['hour']
df_4['year'] = [x[:4] for x in df_4.index.astype('str')]
df_4[['hour', 'dayofweek', 'month', 'holidays', 'day', 'year', 'weekday-hour', 'month-hour']] = df_4[['hour', 'dayofweek', 'month', 'holidays', 'day', 'year', 'weekday-hour', 'month-hour']].astype(np.int32)
df_4

In [ ]:
import numpy as np
df_4.reset_index(inplace=True)
def is_bridge_day(day, holiday, dayofweek):
    if holiday == 1:
        return 7
    elif dayofweek in [5,6] and holiday == 0:
        return dayofweek
    elif dayofweek == 0 and holiday == 0 and df_4.iloc[day+24]['holidays'] == 1:
        return 8
    elif dayofweek == 4 and holiday == 0 and df_4.iloc[day-24]['holidays'] == 1:
        return 8
    else:
        return dayofweek

df_4['day_of_week'] = np.vectorize(is_bridge_day)(df_4.index, df_4['holidays'], df_4['dayofweek'])

In [ ]:
exog_past = [f'y_t-{i}' for i in range(1, 169)]
exog_past.append('psvda_t-24')

exog_fut = [f'load_forecast_t+{i}' for i in range(0, 24)]
exog = exog_past + exog_fut

target = [f'y_t+{i}' for i in range(24)]

calendar = ['hour', 'day_of_week', 'month', 'day', 'year']
df_4.set_index('ds', inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

encoder = LabelEncoder()

for cal in calendar:
  df_4[cal] = encoder.fit_transform(df_4[cal]).astype(np.int32)

In [ ]:
# Assuming your dataset is stored in a DataFrame called df_4

# Determine the indices for splitting the dataset
test_index = len(df_4) - 190*24
validation_index = test_index - 190*24

# Split the dataset into training, validation, and test sets
Y_train_df = df_4.iloc[:validation_index]
Y_val_df = df_4.iloc[validation_index:test_index]
Y_test_df = df_4.iloc[test_index:]

plt.plot(Y_train_df.index, Y_train_df['y_t+0'])
plt.plot(Y_val_df.index, Y_val_df['y_t+0'])
plt.plot(Y_test_df.index, Y_test_df['y_t+0'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_1 = MinMaxScaler()
scaler_2 = MinMaxScaler()
variables = exog + calendar

scaler_1.fit(Y_train_df[exog])
scaler_2.fit(Y_train_df[target])

# training set scaled
Y_train_scaled_exog = pd.DataFrame(scaler_1.transform(Y_train_df[exog]), columns=Y_train_df[exog].columns, index=Y_train_df.index)
Y_train_scaled_df = pd.concat([Y_train_scaled_exog, Y_train_df[calendar]], axis=1)
Y_train_scaled_y = pd.DataFrame(scaler_2.transform(Y_train_df[target]), columns=Y_train_df[target].columns)
# test set scaled
Y_test_scaled_exog = pd.DataFrame(scaler_1.transform(Y_test_df[exog]), columns=Y_test_df[exog].columns, index=Y_test_df.index)
Y_test_scaled_df = pd.concat([Y_test_scaled_exog, Y_test_df[calendar]], axis=1)
Y_test_scaled_y = pd.DataFrame(scaler_2.transform(Y_test_df[target]), columns=Y_test_df[target].columns)

# validation set scaled
Y_val_scaled_exog = pd.DataFrame(scaler_1.transform(Y_val_df[exog]), columns=Y_val_df[exog].columns, index=Y_val_df.index)
Y_val_scaled_df = pd.concat([Y_val_scaled_exog, Y_val_df[calendar]], axis=1)
Y_val_scaled_y = pd.DataFrame(scaler_2.transform(Y_val_df[target]), columns=Y_val_df[target].columns)


# train + validation set scaled
Y_trainval_scaled_df = pd.concat([Y_train_scaled_df, Y_val_scaled_df], axis=0)
Y_trainval_scaled_y = pd.concat([Y_train_scaled_y, Y_val_scaled_y], axis=0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Sample dat
# Assume you have a DataFrame called 'df' containing the relevant data including calendar variables and percentage deviations.

# Define input shapes
num_calendar_features = len(calendar)  # Number of calendar features: hour, day_of_month, day_of_week, holiday, month, quarter
num_exog_features = len(exog)  # Number of past observed percentage deviations to consider as input

# Prepare train input data
calendar_features = Y_train_scaled_df[calendar].values

past_regressors = Y_train_scaled_df[exog].values

target_variable = Y_train_scaled_y[target].values

# Prepare val input data
calendar_features_val = Y_val_scaled_df[calendar].values

past_regressors_val = Y_val_scaled_df[exog].values

target_variable_val = Y_val_scaled_y[target].values

#Prepare test input data
test_calendar_features = Y_test_scaled_df[calendar].values
test_past_regressors = Y_test_scaled_df[exog].values

# training + validation input data
trainval_calendar_features = Y_trainval_scaled_df[calendar].values

trainval_past_regressors = Y_trainval_scaled_df[exog].values

trainval_target = Y_trainval_scaled_y[target].values

In [ ]:
from keras.layers import Concatenate, Dense, Embedding, Flatten, Input, BatchNormalization, Dropout
from keras.layers import TimeDistributed, Reshape
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

def quantile_loss(q, y_true, y_pred):
    error = (y_true - y_pred)
    return tf.reduce_mean(tf.maximum(q * error, (q - 1) * error), axis=-1)

In [ ]:
# Define input layers
hour_input = Input(shape=(1,), name='hour_input')
dow_input = Input(shape=(1,), name='dow_input')
month_input = Input(shape=(1,), name='month_input')
dom_input = Input(shape=(1,), name='dom_input')
year_input = Input(shape=(1,), name='year_input')
weho_input = Input(shape=(1,), name='week_hour_input')
moho_input = Input(shape=(1,), name='month_hour_input')

past_regressors_input = Input(shape=(num_exog_features,), name='past_regressors_input')

# Embedding layer for "hour" variable
hour_embedding_dim = 8
embedded_hour = Embedding(input_dim=24, output_dim=hour_embedding_dim, input_length=1, name='embedding_hour')(hour_input)
flatten_hour = Flatten()(embedded_hour)

# Embedding layer for "day of week" variable
dow_embedding_dim = 3
embedded_dow = Embedding(input_dim=9, output_dim=dow_embedding_dim, input_length=1, name='embedding_dow')(dow_input)
flatten_dow = Flatten()(embedded_dow)

# Embedding layer for "month" variable
month_embedding_dim = 4
embedded_month = Embedding(input_dim=12, output_dim=month_embedding_dim, input_length=1, name='embedding_month')(month_input)
flatten_month = Flatten()(embedded_month)

# Embedding layer for "day of month" variable
dom_embedding_dim = 10
embedded_dom = Embedding(input_dim=31, output_dim=dom_embedding_dim, input_length=1, name='embedding_dom')(dom_input)
flatten_dom = Flatten()(embedded_dom)

# Embedding layer for "year" variable
year_embedding_dim = 3
embedded_year = Embedding(input_dim=6, output_dim=year_embedding_dim, input_length=1, name='embedding_year')(year_input)
flatten_year = Flatten()(embedded_year)


# Concatenate the embedding layers and past_deviations_input
merged_inputs = Concatenate()([flatten_hour,
                               flatten_dow, flatten_month,
                               flatten_dom, flatten_year,
                               past_regressors_input])


def my_probmodel(neurons, activation):

  # Add dense layers
  h = Dense(neurons, activation=activation)(merged_inputs)
  h = Dense(neurons, activation=activation)(h)

  # Create separate output neurons for each time step
  outputs = []
  for i in range(24):
      output = Dense(1, activation='relu', name=f'time_step_{i+1}')(h)
      outputs.append(output)

  # Create the model
  model = keras.Model(inputs=[hour_input,
                            dow_input, month_input,
                            dom_input, year_input,
                            past_regressors_input],
                            outputs=outputs)
  return model


In [ ]:
x=my_probmodel_dist_gaus(160, 'relu')
x.summary()
from keras.utils import plot_model
plot_model(x, rankdir='LR')

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers
import tensorflow_probability as tfp



log_dir='logs'
# Create a dictionary to map embedding layer names to their metadata files
embeddings_metadata = {
    'embedding_hour': 'metadata/metadata_hour.tsv',
    'embedding_dow': 'metadata/metadata_dow.tsv',
    'embedding_month': 'metadata/metadata_month.tsv',
    'embedding_dom': 'metadata/metadata_dom.tsv',
    'embedding_year': 'metadata/metadata_year.tsv',
}

# Initialize the TensorBoard callback with the embeddings metadata
tensorboard_callback = TensorBoard(log_dir=log_dir, embeddings_freq=1, embeddings_metadata=embeddings_metadata)

In [ ]:
trainval_calendar_features = trainval_calendar_features.astype(np.float32)
trainval_past_regressors = trainval_past_regressors.astype(np.float32)
trainval_target = trainval_target.astype(np.float32)

In [ ]:
!pip install keras-tuner

In [ ]:
from keras.layers.normalization import batch_normalization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from tensorflow.keras import regularizers

# Define the hyperparameter search space
def build_model(hp):
  # Define the input layers
    hour_input = Input(shape=(1,), name='hour')
    dow_input = Input(shape=(1,), name='day_of_week')
    month_input = Input(shape=(1,), name='month')
    dom_input = Input(shape=(1,), name='day_of_month')
    year_input = Input(shape=(1,), name='year')


    past_regressors_input = Input(shape=(past_regressors.shape[1],), name='past_regressors')


    # Embedding layer for "hour" variable
    hour_embedding_dim = hp.Int('hour_embedding_dim', min_value=4, max_value=10)
    embedded_hour = Embedding(input_dim=24, output_dim=hour_embedding_dim, input_length=1, name='embedding_hour')(hour_input)
    flatten_hour = Flatten()(embedded_hour)

    # Embedding layer for "day of week" variable
    dow_embedding_dim = hp.Int('dow_embedding_dim', min_value=3, max_value=5)
    embedded_dow = Embedding(input_dim=9, output_dim=dow_embedding_dim, input_length=1, name='embedding_dow')(dow_input)
    flatten_dow = Flatten()(embedded_dow)

    # Embedding layer for "month" variable
    month_embedding_dim = hp.Int('month_embedding_dim', min_value=3, max_value=6)
    embedded_month = Embedding(input_dim=12, output_dim=month_embedding_dim, input_length=1, name='embedding_month')(month_input)
    flatten_month = Flatten()(embedded_month)

    # Embedding layer for "day of month" variable
    dom_embedding_dim = hp.Int('dom_embedding_dim', min_value=6, max_value=14)
    embedded_dom = Embedding(input_dim=31, output_dim=dom_embedding_dim, input_length=1, name='embedding_dom')(dom_input)
    flatten_dom = Flatten()(embedded_dom)

    # Embedding layer for "year" variable
    year_embedding_dim = hp.Int('year_embedding_dim',  min_value=2, max_value=4)
    embedded_year = Embedding(input_dim=6, output_dim=year_embedding_dim, input_length=1, name='embedding_year')(year_input)
    flatten_year = Flatten()(embedded_year)


    # Concatenate the embedding layers and past_deviations_input
    merged_inputs = Concatenate()([flatten_hour,
                                  flatten_dow, flatten_month,
                                  flatten_dom, flatten_year,
                                  past_regressors_input])

    quantiles = [0.1, 0.5, 0.9]  # Example quantiles
    num_models=len(quantiles)
    predictions = [] # store predicitons
    model = my_probmodel_dist_gaus(160, 'relu')

    # Define the hidden layers
    units_1 = hp.Int('units_1', min_value=32, max_value=512, step=32)
    units_2 = hp.Int('units_2', min_value=32, max_value=512, step=32)

    hidden_1 = Dense(units=160, activation='relu')(merged_inputs)
    dropout = hp.Choice('dropout', values=[True, False])
    if dropout == True:
        dropout_layer = Dropout(rate=hp.Choice('dropout_value', values=[0.0, 0.2, 0.4]))(hidden_1)
    hidden_2 = Dense(units=160, activation='relu')(hidden_1)
    else:
        hidden_2 = Dense(units=units_1, activation='relu')(hidden_1)
    # Create separate output neurons for each time step
    #outputs = []
    for i in range(24):
        output = Dense(1, activation='linear', name=f'time_step_{i+1}')(hidden_2)
        outputs.append(output)


    # Create the model
    model = keras.Model(inputs=[hour_input,
                                  dow_input, month_input,
                                  dom_input, year_input,
                                  past_regressors_input],
                                  outputs=output)
    # Compile the model with negative log likelihood as the loss function
    model.compile(optimizer=Adam(),  loss=lambda y_true, y_pred: negative_log_likelihood(y_true, y_pred))


    return model

# Instantiate the tuner
tuner = RandomSearch(build_model,
                     objective='val_loss',
                     max_trials=10,
                     executions_per_trial=1,
                     directory='my_dir',
                     project_name='my_proj',
                     overwrite = True)

# Train the tuner
tuner.search(x=[calendar_features[:, 0], calendar_features[:, 1], calendar_features[:, 2],
                        calendar_features[:, 3], calendar_features[:, 4],
                       past_regressors],
                    y=[target_variable[:, i] for i in range(24)],

              validation_data=([calendar_features_val[:, 0], calendar_features_val[:, 1], calendar_features_val[:, 2],
                       calendar_features_val[:, 3], calendar_features_val[:, 4],
                       past_regressors_val],
                    [target_variable_val[:, i] for i in range(24)]),

              epochs=10, batch_size=32)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers

log_dir='logs'
# Create a dictionary to map embedding layer names to their metadata files
embeddings_metadata = {
    'embedding_hour': 'metadata/metadata_hour.tsv',
    'embedding_dow': 'metadata/metadata_dow.tsv',
    'embedding_month': 'metadata/metadata_month.tsv',
    'embedding_dom': 'metadata/metadata_dom.tsv',
    'embedding_year': 'metadata/metadata_year.tsv',
}

# Initialize the TensorBoard callback with the embeddings metadata
tensorboard_callback = TensorBoard(log_dir=log_dir, embeddings_freq=1, embeddings_metadata=embeddings_metadata)

# Define the input layers
hour_input = Input(shape=(1,), name='hour')
dow_input = Input(shape=(1,), name='day_of_week')
month_input = Input(shape=(1,), name='month')
dom_input = Input(shape=(1,), name='day_of_month')
year_input = Input(shape=(1,), name='year')


past_regressors_input = Input(shape=(past_regressors.shape[1],), name='past_regressors')

# Embedding layer for "hour" variable
hour_embedding_dim = best_hps.values['hour_embedding_dim']
embedded_hour = Embedding(input_dim=24, output_dim=hour_embedding_dim, input_length=1, name='embedding_hour')(hour_input)
flatten_hour = Flatten()(embedded_hour)

# Embedding layer for "day of week" variable
dow_embedding_dim = best_hps.values['dow_embedding_dim']
embedded_dow = Embedding(input_dim=9, output_dim=dow_embedding_dim, input_length=1, name='embedding_dow')(dow_input)
flatten_dow = Flatten()(embedded_dow)

# Embedding layer for "month" variable
month_embedding_dim = best_hps.values['month_embedding_dim']
embedded_month = Embedding(input_dim=12, output_dim=month_embedding_dim, input_length=1, name='embedding_month')(month_input)
flatten_month = Flatten()(embedded_month)

# Embedding layer for "day of month" variable
dom_embedding_dim = best_hps.values['dom_embedding_dim']
embedded_dom = Embedding(input_dim=31, output_dim=dom_embedding_dim, input_length=1, name='embedding_dom')(dom_input)
flatten_dom = Flatten()(embedded_dom)

# Embedding layer for "year" variable
year_embedding_dim = best_hps.values['year_embedding_dim']
embedded_year = Embedding(input_dim=6, output_dim=year_embedding_dim, input_length=1, name='embedding_year')(year_input)
flatten_year = Flatten()(embedded_year)



# Concatenate the embedding layers and past_deviations_input
merged_inputs = Concatenate()([flatten_hour,
                                flatten_dow, flatten_month,
                              flatten_dom, flatten_year,
                               past_regressors_input])

quantiles = [0.1, 0.5, 0.9]  # Example quantiles
num_models=len(quantiles)
predictions = [] # store predicitons

for i, q in enumerate(quantiles):
    # Add dense layers
    hidden_1 = Dense(best_hps.values['units_1'], activation='relu')(merged_inputs)
    hidden_2 = Dense(best_hps.values['units_2'], activation='relu')(hidden_1)

    # Create separate output neurons for each time step
    outputs = []
    for i in range(24):
        output = Dense(1, activation='linear', name=f'time_step_{i+1}')(hidden_2)
        outputs.append(output)

    # Create the model
    model = keras.Model(inputs=[hour_input,
                                dow_input, month_input,
                              dom_input, year_input,
                               past_regressors_input],
                              outputs=outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_hps.values['learning_rate'])
    model.compile(loss=lambda y_true, y_pred: quantile_loss(q, y_true, y_pred), optimizer='adam')

    # Train the model on the combined training and validation sets
    model.fit(x=[trainval_calendar_features[:, 0], trainval_calendar_features[:, 1], trainval_calendar_features[:, 2],
                          trainval_calendar_features[:, 3], trainval_calendar_features[:, 4],
                          trainval_past_regressors],
                        y=[trainval_target[:, i] for i in range(24)],

              validation_data=([calendar_features_val[:, 0], calendar_features_val[:, 1], calendar_features_val[:, 2],
                       calendar_features_val[:, 3], calendar_features_val[:, 4],
                       past_regressors_val],
                    [target_variable_val[:, i] for i in range(24)]),

                  epochs=10, batch_size=32,
                  callbacks=[tensorboard_callback],
                  verbose=2)

    # Predict on the test data
    prediction = model.predict([test_calendar_features[:, 0], test_calendar_features[:, 1],
                                    test_calendar_features[:, 2], test_calendar_features[:, 3],
                                    test_calendar_features[:, 4],
                                    test_past_regressors])
    predictions.append(prediction)

predictions = np.asarray(predictions)
preds = np.squeeze(predictions)
preds

In [ ]:
rescaled_all = pd.DataFrame()

for index, element in enumerate(preds_2):
    df_predictions = pd.DataFrame()
    time_step_predictions = {}
    # Access predictions for each time step
    for i in range(0, 24):
        time_step = i + 1
        time_step_prediction = element[i]

        # Store the predictions in the dictionary
        time_step_predictions[f'time_step_{time_step}_q{str(index+1)}'] = pd.Series(time_step_prediction.flatten())

        # Create a DataFrame from the dictionary
        df_predictions = pd.concat(time_step_predictions, axis=1)

    df_predictions_rescaled = pd.DataFrame(scaler_2.inverse_transform(df_predictions), columns=df_predictions.columns)
    rescaled_all = pd.concat([rescaled_all, df_predictions_rescaled], axis=1)

test_df = Y_test_df[target]
test_df.reset_index(inplace=True)

merged_df = pd.concat([test_df, rescaled_all], axis=1)
merged_df

In [ ]:
# Determine the total number of rows in the DataFrame
total_rows = merged_df.shape[0]

df_final = pd.DataFrame()
# Iterate over the indices in steps of 24
for start_index in range(0, total_rows, 24):
    # Calculate the end index for each group
    end_index = start_index + 24

    # Extract the group of rows based on the start and end indices
    group_df = merged_df.iloc[start_index:end_index]

    date = []
    actual = []
    pred = []
    pred_q1 = []
    pred_q2 = []
    pred_q3 = []
    pred_q4 = []
    pred_q5 = []
    total_rows_group = group_df.shape[0]
    for i in range(0, total_rows_group):
        # Calculate the end index for each group
        date.append(group_df['ds'].iloc[i])
        actual.append(group_df['y_t+0'].iloc[i])
        for index, element in enumerate(preds):
          if index == 0:
            pred_q1.append(group_df[f'time_step_{i+1}_q{str(index+1)[-1]}'].iloc[0])

          elif index == 1:
            pred_q2.append(group_df[f'time_step_{i+1}_q{str(index+1)[-1]}'].iloc[0])

          elif index == 2:
            pred_q3.append(group_df[f'time_step_{i+1}_q{str(index+1)[-1]}'].iloc[0])

          elif index == 3:
            pred_q4.append(group_df[f'time_step_{i+1}_q{str(index+1)[-1]}'].iloc[0])

          elif index == 4:
            pred_q5.append(group_df[f'time_step_{i+1}_q{str(index+1)[-1]}'].iloc[0])

    df_group = pd.DataFrame(list(zip(date, actual, pred_q1, pred_q2, pred_q3, pred_q4, pred_q5 )),
                            columns=['date', 'actual', 'pred_q1', 'pred_q2', 'pred_q3', 'pred_q4', 'pred_q5'])


    df_final = pd.concat([df_final, df_group], axis=0)

df_final

In [ ]:
from neuralforecast.losses.numpy import mae, mape

df_final['year_month'] = [x[:7] for x in df_final.date.astype('str')]

plt.figure(figsize=(20,5))
plt.plot(df_final['date'], df_final['actual'], label='True', c='black')
plt.plot(df_final['date'], df_final['pred_q3'], c='steelblue', label='Pred')

plt.fill_between(x=df_final['date'],
                        y1=df_final['pred_q1'], y2=df_final['pred_q5'],
                        alpha=0.6, label='level 90', color='steelblue')

plt.fill_between(x=df_final['date'],
                        y1=df_final['pred_q2'], y2=df_final['pred_q4'],
                        alpha=0.6, label='level 80', color='steelblue')

plt.legend()
plt.plot()

print('MAE: ', mae(df_final['actual'], df_final['pred_q2']))
print('MAPE: ', mape(df_final['actual'], df_final['pred_q2']))

In [ ]:
def winkler_score(y_true, y_lower, y_upper, alpha=0.1):
    delta = y_upper - y_lower

    score = np.where((y_true >= y_lower) & (y_true <= y_upper), delta, 0)
    score += np.where(y_true < y_lower, delta + (2 / alpha) * (y_lower - y_true), 0)
    score += np.where(y_true > y_upper, delta + (2 / alpha) * (y_true - y_upper), 0)

    return score.mean()

def pi_width(y_lower, y_upper):
    delta = y_upper - y_lower
    return delta.mean()

def coverage_error(y_true, y_lower, y_upper, alpha):
    coverage_prob = (y_lower <= y_true) & (y_true <= y_upper)
    return np.abs(coverage_prob.mean() - alpha)

def pinball(y_true, y_quantile, alpha):
    pinball_loss = np.maximum((y_true - y_quantile) * alpha, (y_quantile - y_true) * (1 - alpha))
    return np.mean(pinball_loss)

def unconditional_coverage_score(y_true, y_lower, y_upper):
    coverage_scores = ((y_true >= y_lower) & (y_true <= y_upper)).astype(int)
    return np.mean(coverage_scores)

In [ ]:
from neuralforecast.losses.numpy import mae, mape, smape
from sklearn.metrics import mean_pinball_loss
df_final['year_month'] = [x[:7] for x in df_final.index.astype('str')]

results_df = pd.DataFrame()

for month in df_final['year_month'].drop_duplicates():
    df_err = df_final[df_final['year_month']==month]
    plt.figure(figsize=(20,5))
    plt.plot(df_err.index, df_err['actual'], label='True', c='black')
    plt.plot(df_err.index, df_err['pred_q3'], c='steelblue', label='Pred')

    plt.fill_between(x=df_err.index,
                        y1=df_err['pred_q1'], y2=df_err['pred_q5'],
                        alpha=0.3, label='level 90', color='steelblue')
    plt.fill_between(x=df_err.index,
                        y1=df_err['pred_q2'], y2=df_err['pred_q4'],
                        alpha=0.3, label='level 90', color='steelblue')

    plt.legend()
    plt.plot()

    my_results = {'date': pd.to_datetime(month),
                  'MAE': round(mae(df_err['actual'], df_err['pred_q3']), 2),
                  'sMAPE': round(smape(df_err['actual'], df_err['pred_q3'])*100, 2),
                  'Pinball Q90': round(mean_pinball_loss(df_err['actual'], df_err['pred_q4'], alpha=0.9), 2),
                  'UC': round(unconditional_coverage_score(df_err['actual'], df_err['pred_q2'], df_err['pred_q4'])*100, 2),
                  'Winkler score': round(winkler_score(df_err['actual'], df_err['pred_q2'], df_err['pred_q4'], alpha=0.2), 2),
                  'PI width': round(pi_width(df_err['pred_q2'], df_err['pred_q4']), 2)}

    results = pd.DataFrame([my_results])
    results['sMAPE'] = results['sMAPE'].astype(str) + '%'
    results['ACE'] = round((80 - results['UC']), 2).astype(str) + '%'
    results['UC'] = results['UC'].astype(str) + '%'
    results['Penalty'] = results['Winkler score'] - results['PI width']
    results_df = pd.concat([results_df, results], axis=0, ignore_index=True)
results_df